In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy
import pickle
import xgboost as xgb
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.feature_selection import SelectPercentile, f_regression
from time import time, strftime, localtime

###  Load data

In [3]:
train_df = pd.read_csv("../dataset/train.csv")
test_df = pd.read_csv("../dataset/test.csv")
submit_test_df = pd.read_csv("../dataset/submit_test.csv")

### Split data & process missing value

Fill nan with:
* "0.0"
* median()
* mean()

In [162]:
# Fill nan with "0.0"
train_fillna_df = train_df.iloc[:,1:-1].fillna(value=0.0)
test_fillna_df = test_df.iloc[:,1:].fillna(value=0.0)
all_fillna_df = pd.concat((train_fillna_df, test_fillna_df), axis = 0)

# Fill nan with "median"
#train_fillna_df = train_df.iloc[:,1:-1].fillna(train_df.iloc[:,1:-1].median())
#test_fillna_df = test_df.iloc[:,1:].fillna(test_df.iloc[:,1:].median())
#all_fillna_df = pd.concat((train_fillna_df, test_fillna_df), axis = 0)

# Seperate features fillna
#train_noIDnoLabel_df = train_df.iloc[:,1:-1]
#test_noID_df = test_df.iloc[:,1:]
#all_df = pd.concat((train_noIDnoLabel_df, test_noID_df), axis = 0)

label_name = "total_price"
label_df = pd.DataFrame(train_df[label_name])

In [140]:
#temp_df = all_fillna_df.copy()
temp_df = all_df.copy()

### Process miss value

features:
* village_income_median 所在里年收入中位數：以 "city+town" 為 key 填入平均值，如無值則以 "city" 為 key 填入平均值

In [141]:
count = 0
VIM_mean = 0.0
t0 = time()

# Fill nan with the mean of "city+town"
for index, row in all_df.iloc[:][(all_df["village_income_median"].isnull())].iterrows():
    count += 1
    VIM_mean = all_df["village_income_median"][(all_df["city"] == row["city"]) & (all_df["town"] == row["town"])].mean()
    # If the mean of "city+town" is null or zero, fill nan with the mean of "city"
    if math.isnan(VIM_mean) or VIM_mean == 0:
        VIM_mean = all_df["village_income_median"][all_df["city"] == row["city"]].mean()
    temp_df["village_income_median"][(temp_df["village_income_median"].isnull()) & (temp_df["city"] == row["city"]) & (temp_df["town"] == row["town"])] = VIM_mean

print('spent time: %0.3fs' % (time() - t0), strftime("%Y-%m-%d %H:%M:%S", localtime()))
print ("count:", count)

C:\Users\me\Anaconda3\envs\tensorflow-gpu-py3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


spent time: 63.623s 2019-07-12 17:59:29
count: 1326


In [142]:
# Fill nan with "0.0"
all_fillna_df = temp_df.fillna(value=0.0)

### Drop no need features

In [362]:
all_process_df = all_fillna_df.drop(
    ["parking_area","parking_price"], axis=1)
print("all_process_df.shape:", all_process_df.shape)

all_process_df.shape: (70000, 231)


### Create features
* [house_age 購買時的屋齡]：以10年為一個區間，((成交日 - 完工日) / 3650) ((txn_dt - building_complete_dt) / 3650)
* [txn_year 交易年]：the year of trade，成交日 / 365, txn_dt / 365
* [floor_of_building 樓層高度比例]：floor of building，交易樓層 / 總樓層(txn_floor / total_floor)，如 txn_floor 為 0，代表建物不為一層，以總樓層帶入
* [sales_per_unit_area 坪效]：一坪土地可蓋出多少坪可銷售面積，(建物面積 / 土地面積)(building_area / land_area)
* [power_pos_rate 強化正相關率]：將會提高房價的因素加總，再減會減低房價的因素，(所在縣市出生率 + 所在縣市結婚率 - 所在縣市死亡率) (born_rate + marriage_rate - death_rate)
* [high_edu_rate 各縣市高學歷率]：各縣市的高學歷加總比率(大學含以上)
* [low_edu_rate 各縣市低學歷率]：各縣市的低學歷加總比率(專科+高中含以下)
* [is_apartment_first_top 是否為公寓的一樓跟頂樓]：公寓的一樓跟頂樓最貴
* [is_0_3_high_floor 是否為電梯大樓、華廈的高樓層(7樓以上)]：樓層越高越貴
* [_0_3_high_floor_rate 電梯大樓、華廈的高樓層高度比率(7樓以上)]：樓層越高越貴

In [430]:
# [house_age: the age of house on purchased date, ((成交日 - 完工日) / 3650) ((txn_dt - building_complete_dt) / 3650)]
all_createCol_df = all_fillna_df.assign(house_age = round((all_fillna_df["txn_dt"] - all_fillna_df["building_complete_dt"]) / 3650, 0))

# [txn_year: the year of trade, 成交日 / 365, txn_dt / 365]
all_createCol_df = all_createCol_df.assign(txn_year = round((all_fillna_df["txn_dt"] / 365), 0))

# [floor_of_building: floor of building, 交易樓層 / 總樓層(txn_floor / total_floor)]，如 txn_floor 為 0，代表建物不為一層，以總樓層帶入
all_createCol_df = all_createCol_df.assign(floor_of_building = all_fillna_df["txn_floor"] / all_fillna_df["total_floor"])
all_createCol_df["floor_of_building"][all_createCol_df["txn_floor"] == 0] = all_fillna_df["total_floor"]

inf = float("inf")
# [sales_per_unit_area 坪效: 一坪土地可蓋出多少坪可銷售面積, (建物面積 / 土地面積), (building_area / land_area)]
all_createCol_df = all_createCol_df.assign(sales_per_unit_area = round(all_createCol_df["building_area"] / all_createCol_df["land_area"], 1))
all_createCol_df["sales_per_unit_area"][all_createCol_df["sales_per_unit_area"] == inf] = all_createCol_df["sales_per_unit_area"].median()

# [power_pos_rate 強化正相關率: 將會提高房價的因素加總，再減會減低房價的因素, (所在縣市出生率 + 所在縣市結婚率 - 所在縣市死亡率), (born_rate + marriage_rate - death_rate)]
all_createCol_df = all_createCol_df.assign(power_pos_rate = all_createCol_df["born_rate"] + all_createCol_df["marriage_rate"] - all_createCol_df["death_rate"])
#all_createCol_df = all_fillna_df.assign(power_pos_rate = all_fillna_df["born_rate"] + all_fillna_df["marriage_rate"] - all_fillna_df["death_rate"])

# [high_edu_rate 各縣市高學歷率: 各縣市的高學歷加總比率(大學含以上)]
#all_createCol_df = all_createCol_df.assign(high_edu_rate = round(all_createCol_df["doc_rate"] + all_createCol_df["master_rate"] + all_createCol_df["bachelor_rate"], 3))

# [low_edu_rate 各縣市低學歷率: 各縣市的低學歷加總比率(專科+高中含以下)]
#all_createCol_df = all_createCol_df.assign(low_edu_rate = round(all_createCol_df["jobschool_rate"] + all_createCol_df["highschool_rate"] + all_createCol_df["junior_rate"] + all_createCol_df["elementary_rate"], 3))

# [is_apartment_first_top 是否為公寓的一樓跟頂樓: 公寓的一樓跟頂樓最貴]
all_createCol_df = all_createCol_df.assign(is_apartment_first_top = 0.0)
all_createCol_df["is_apartment_first_top"][(all_createCol_df["building_type"] == 1) & ((all_createCol_df["txn_floor"] == 1) |  (all_createCol_df["txn_floor"] == all_createCol_df["total_floor"]))] = 1

# [is_0_3_high_floor 是否為電梯大樓、華廈的高樓層(7樓以上): 樓層越高越貴]
#all_createCol_df = all_createCol_df.assign(is_0_3_high_floor = 0.0)
#all_createCol_df["is_0_3_high_floor"][((all_createCol_df["building_type"] == 0) | (all_createCol_df["building_type"] == 3)) & (all_createCol_df["txn_floor"] >= 7)] = 1

# [_0_3_high_floor_rate 電梯大樓、華廈的高樓層高度比率(7樓以上): 樓層越高越貴]
#all_createCol_df = all_createCol_df.assign(_0_3_high_floor_rate = 0.0)
#all_createCol_df["_0_3_high_floor_rate"][((all_createCol_df["building_type"] == 0) | (all_createCol_df["building_type"] == 3)) & (all_createCol_df["txn_floor"] >= 7)] = (all_fillna_df["txn_floor"] / all_fillna_df["total_floor"])

C:\Users\me\Anaconda3\envs\tensorflow-gpu-py3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


#### Check data

In [434]:
all_createCol_df[["is_apartment_first_top"]].head()

,is_apartment_first_top
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0


In [435]:
all_createCol_df.shape

(70000, 239)

In [436]:
#all_process_df = all_df.copy()
#all_process_df = all_selection_df.copy()
#all_process_df = all_fillna_df.copy()
all_process_df = all_createCol_df.copy()

### Normalization
* min max scalar: normalize with training data & testing data

In [437]:
# [No-ohe version]
# preserve column name after scaler
min_max_scaler = preprocessing.MinMaxScaler()
mms_features_df = pd.DataFrame(min_max_scaler.fit_transform(all_process_df))

## Split to train & test data
train_num = train_df.shape[0]
#train_num = train_remove_outlier_df.shape[0]
mms_train_process_df = mms_features_df.iloc[:train_num,:]
mms_test_process_df = mms_features_df.iloc[train_num:,:]

print("mms_train_process_df.shape:", mms_train_process_df.shape)
print("mms_test_process_df.shape:", mms_test_process_df.shape)

mms_train_process_df.shape: (60000, 239)
mms_test_process_df.shape: (10000, 239)


### Shuffle data & Split data
* random_state = 7

In [438]:
#shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_df), axis=1))
# No min max scalar
#shuff_train_all = shuffle(pd.concat((train_process_df, label_df), axis=1))
# Drop unimportant features
#shuff_train_all = shuffle(pd.concat((temp_train_df, label_df), axis=1))
# min max
shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_df), axis=1), random_state=7)
#shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_df), axis=1))
# Drop & min max & remove outliers
#shuff_train_all = shuffle(pd.concat((mms_train_process_df, label_remove_outliers_df), axis=1), random_state=0)
# Drop & remove outliers
#shuff_train_all = shuffle(pd.concat((train_drop_df, label_remove_outliers_df), axis=1), random_state=0)


# process train and validation num
tv_num = round(shuff_train_all.shape[0] * 0.85)

### Split data
# training data
X_train = shuff_train_all.iloc[:tv_num,:-1]
y_train = shuff_train_all.iloc[:tv_num,-1:]

print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)

# validation data
X_val = shuff_train_all.iloc[tv_num:,:-1]
y_val = shuff_train_all.iloc[tv_num:,-1:]

print("X_val.shape:", X_val.shape)
print("y_val.shape:", y_val.shape)

X_train.shape: (51000, 239)
y_train.shape: (51000, 1)
X_val.shape: (9000, 239)
y_val.shape: (9000, 1)


### Transfter into data matrix

In [439]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalidation = xgb.DMatrix(X_val, label=y_val)
#dtest = xgb.DMatrix(mms_test_process_df)
# No min max scaler
#dtest = xgb.DMatrix(test_process_df)
# Drop unimportant features
#dtest = xgb.DMatrix(temp_test_df)
# Drop & min max
dtest = xgb.DMatrix(mms_test_process_df)
# Drop
#dtest = xgb.DMatrix(test_drop_df)

### Model
* XGBoost
    * booster: gradient boosting tree
    * objective: regression with squared loss
    * eval_metric: RMSE
    * learning rate: 0.01

In [ ]:
param = {'objective': 'reg:squarederror', 'colsample_bytree': 0.8, 'subsample': 0.5, 'max_depth': 10, 'eta': 0.01, 
        'min_child_weight': 0.3, 'reg_alpha': 0.01, 'reg_lambda': 1, 'gamma': 0.0001, 'verbosity': 1, 'random_state': 7, 
        'nthread': -1, 'tree_method': 'gpu_hist', 'booster': 'gbtree'}
param['eval_metric'] = ['rmse']

### Training model
print('開始時間：', strftime("%Y-%m-%d %H:%M:%S", localtime()))
t0 = time()

#evallist = [(dtrain, 'train'), (dvalidation, 'eval')]
evallist = [(dvalidation, 'eval'), (dtrain, 'train')]
num_round = 200000
#bst = xgb.train(param, dtrain, num_round, evallist)
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10)

print('spent time: %0.3fs' % (time() - t0), strftime("%Y-%m-%d %H:%M:%S", localtime()))

### Predict
print('開始時間：', strftime("%Y-%m-%d %H:%M:%S", localtime()))
t0 = time()

xgb_pred = bst.predict(dtest)
y_test = bst.predict(dtrain)

print('spent time: %0.3fs' % (time() - t0), strftime("%Y-%m-%d %H:%M:%S", localtime()))

# validation
RMSE = np.sqrt(mean_squared_error(y_train, y_test))
print("{:,}".format(RMSE.round(4)))

### Validation
* RMSE on training data
* RMSE on testing data

In [ ]:
RMSE_train = np.sqrt(mean_squared_error(y_train, y_test))
print('RMSE_train:', "{:,}".format(RMSE_train.round(4)))

y_val_test = bst.predict(dvalidation)
RMSE_val = np.sqrt(mean_squared_error(y_val, y_val_test))
print('RMSE_val:', "{:,}".format(RMSE_val.round(4)))

### Describe data

In [453]:
print("y_train:", pd.DataFrame(y_train).describe())
print("y_test:", pd.DataFrame(y_test).describe())
print("xgb_pred:", pd.DataFrame(xgb_pred).describe(), "\n")
print("number of xgb_pred are negative:", len(xgb_pred[xgb_pred < 0]))

y_train:         total_price
count  5.100000e+04
mean   1.284103e+07
std    5.108065e+07
min    2.261495e+05
25%    2.436585e+06
50%    5.240482e+06
75%    1.123932e+07
max    4.357039e+09
y_test:                   0
count  5.100000e+04
mean   1.284045e+07
std    5.101806e+07
min   -5.353824e+06
25%    2.635457e+06
50%    5.391494e+06
75%    1.117924e+07
max    4.355997e+09
xgb_pred:                   0
count  1.000000e+04
mean   1.319855e+07
std    4.696391e+07
min   -2.750140e+07
25%    2.580645e+06
50%    5.396035e+06
75%    1.103688e+07
max    2.038183e+09 

number of xgb_pred are negative: 36


### Save model

In [ ]:
with open('../model/xgb_tree_1.pkl', 'wb') as f:
    pickle.dump(bst, f)

### Submission
自動產出數字最新的 submit 檔

In [455]:
import os
import re

# 取得 submit 檔最大值的下一個數字
def getMaxFileNum():
    max = 0
    for f in os.listdir("../submit"):
        fileNum = re.findall('\d+', f)
        if len(fileNum):
            if int(fileNum[0]) > max:
                max = int(fileNum[0])
    return str(max + 1)

filePath = "../submit/submit_test_" + getMaxFileNum() + ".csv"

pred_df = pd.DataFrame(np.array(xgb_pred), columns=["total_price"])
ans_df = pd.merge(submit_test_df["building_id"].to_frame(), pred_df, left_index=True, right_index=True, how="outer")
ans_df.to_csv(filePath,sep=",",index=False,encoding="UTF-8")

print('filePath:', filePath)

filePath: ../submit/submit_test_72.csv
